In [1]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
#from qiskit.tools.jupyter import *
#from qiskit.visualization import *
# Loading your IBM Q account(s)
#provider = IBMQ.load_account()


# Import libraries
from qiskit import *
from qiskit.circuit import QuantumRegister, ClassicalRegister, QuantumCircuit, Qubit, Clbit, Gate, Parameter, InstructionSet
from qiskit.aqua.components.oracles import Oracle, TruthTableOracle
import numpy as np
import math
from copy import deepcopy

qiskit.__qiskit_version__

import logging

from sympy.combinatorics.graycode import GrayCode
from math import pi

from qiskit.aqua.utils.controlled_circuit import apply_cu3
from qiskit.aqua import AquaError

# Protein folding with Metropolis and Quantum Walks

Work plan

1.    Program Metropolis in superposition using Qiskit. From arXiv:1910.01659
2.    Figure out how to rotate the structure of the proteins. Perhaps use psi4 to figure out the energy of each configuration
3.    Prove it with usual Metropolis and Tripeptides: https://en.wikipedia.org/wiki/Tripeptide

Since we are going to use IBM simulator, we can use up to 32 qubits!

The first thing we need to do is to calculate the registers we need in order to be able to perform the metropolis algorithm:

For example, for a tripeptide (n=3) we have 2(n-1) angles

Coin: 1 to allow or decline move

Move: 2 moves on 4 angles: 2 to identify angle 1 to identify if we increase or decrease it

State: 32 divisions (5 qubits) for each angle 20 qubits = 4 angles * 5 qubits/angle

Ancillas: 8 qubits

Total: 32 qubits, maximum

----
For a dipeptide we have 2 angles

Coin: 1 to allow or decline move

Move: 2 moves on 2 angles: 1 to identify angle 1 to identify if we increase or decrease it

State:  2 angles * 6 qubits/angle

Ancillas: 10 qubits

Total: 32 qubits, maximum

Here we fix the parameters

In [2]:
# For dipeptide
# Indicate number of precision bits
n_precision_bits = 3
n_ancilla_bits = 4 #For the oracle

# I'm unsure about whether to put this here, since it is only necessary to define the size of 
'''
# State definition. 32 divisions for each of the four angles. All angles range from 0 to 2pi
angle_phi = QuantumRegister(n_precision_bits, name = 'angle_phi')
angle_psi = QuantumRegister(n_precision_bits, name = 'angle_psi') 

# Move proposal
move_id = QuantumRegister(1, name = 'move_id') #Which angle are we modifying
move_value = QuantumRegister(1, name = 'move_value') #0 -> decrease the angle. 1-> increase it

# Coin
coin = QuantumRegister(1, name = 'coin')

# Ancillas
ancilla = QuantumRegister(n_ancilla_bits, name = 'ancilla')

# Circuit
qc = QuantumCircuit(angle_phi,angle_psi,move_id,move_value,coin,ancilla)
'''

"\n# State definition. 32 divisions for each of the four angles. All angles range from 0 to 2pi\nangle_phi = QuantumRegister(n_precision_bits, name = 'angle_phi')\nangle_psi = QuantumRegister(n_precision_bits, name = 'angle_psi') \n\n# Move proposal\nmove_id = QuantumRegister(1, name = 'move_id') #Which angle are we modifying\nmove_value = QuantumRegister(1, name = 'move_value') #0 -> decrease the angle. 1-> increase it\n\n# Coin\ncoin = QuantumRegister(1, name = 'coin')\n\n# Ancillas\nancilla = QuantumRegister(n_ancilla_bits, name = 'ancilla')\n\n# Circuit\nqc = QuantumCircuit(angle_phi,angle_psi,move_id,move_value,coin,ancilla)\n"

## Helper gates

In [3]:
def sum1(circuit,qubit_string,control,start,end):
    '''
    Outputs:
    Sums register 1 and 2 (1 qubit) in register 1. Tested ok.
    
    Input:
    circuit: QuantumCircuit with registers qubit_string, control, ancilla
    
    qubit_string: QuantumRegister
    
    control: Qubit. Use ancilla[0] or similar
    
    start: Qubit. Use ancilla[1] or similar
    end: Qubit. Use ancilla[2] or similar
    '''
    n_qubits = qubit_string.size     # calculate n_qubits
    circuit.cx(control,end) # iff control = 1, end = 1
    circuit.x(start)
    circuit.cx(control,start) # iff control = 1, start = 0
    
    for i in range(n_qubits+1): #Don't need to add control, since start already does that work
        '''
        Next thing we analise if all qubits to the right have value 1, 
        and save it in the current qubit and start
        '''
        if i > 0:
            # For i = 0, there is only the start to worry about
            circuit.mcrx(theta = pi, q_controls = [qubit_string[j] for j in range(n_qubits-i)]+[end], q_target = qubit_string[n_qubits-i])
        circuit.mcrx(theta = pi, q_controls = [qubit_string[j] for j in range(n_qubits-i)]+[end], q_target = start)

        '''
        Next, controlling on the current qubit and start, we change all the following qubits to 0.
        We have to control with qubit_string[n_qubit]
        '''
        if i == 0:
            for j in range(n_qubits-i):
                circuit.ccx(control,start,qubit_string[j])
            circuit.ccx(control,start,end)
        elif i == n_qubits:
            circuit.mcrx(theta = pi, q_controls = [control,qubit_string[n_qubits-i],start], q_target = end)
        else:
            for j in range(n_qubits-i):            
                circuit.mcrx(theta = pi, q_controls = [control,qubit_string[n_qubits-i],start], q_target = qubit_string[j])
            circuit.mcrx(theta = pi, q_controls = [control,qubit_string[n_qubits-i],start], q_target = end)
    circuit.x(start)
    
    
'''
# Validation circuit. ------------------------------------------
qubit_string = QuantumRegister(4)
#control = QuantumRegister(1)
ancilla = QuantumRegister(3)

bit_string = ClassicalRegister(4)
#ccontrol = ClassicalRegister(1)
cancilla = ClassicalRegister(3)

qc = QuantumCircuit(qubit_string, ancilla, bit_string, cancilla)

# Initialize state
qc.x(ancilla[0])
#qc.x(qubit_string)

# Gate
sum1(qc,qubit_string,ancilla[0],ancilla[1],ancilla[2])

# Readout
#Measure
qc.measure(qubit_string,bit_string)

qc.measure(ancilla[0],cancilla[0])
qc.measure(ancilla[1],cancilla[1])
qc.measure(ancilla[2],cancilla[2])

# Use Aer's qasm_simulator
backend_sim = Aer.get_backend('qasm_simulator')

# Execute the circuit on the qasm simulator.
# We've set the number of repeats of the circuit
# to be 1024, which is the default.
job_sim = execute(qc, backend_sim, shots=1024)

# Grab the results from the job.
result_sim = job_sim.result()

counts = result_sim.get_counts(qc)

counts'''

"\n# Validation circuit. ------------------------------------------\nqubit_string = QuantumRegister(4)\n#control = QuantumRegister(1)\nancilla = QuantumRegister(3)\n\nbit_string = ClassicalRegister(4)\n#ccontrol = ClassicalRegister(1)\ncancilla = ClassicalRegister(3)\n\nqc = QuantumCircuit(qubit_string, ancilla, bit_string, cancilla)\n\n# Initialize state\nqc.x(ancilla[0])\n#qc.x(qubit_string)\n\n# Gate\nsum1(qc,qubit_string,ancilla[0],ancilla[1],ancilla[2])\n\n# Readout\n#Measure\nqc.measure(qubit_string,bit_string)\n\nqc.measure(ancilla[0],cancilla[0])\nqc.measure(ancilla[1],cancilla[1])\nqc.measure(ancilla[2],cancilla[2])\n\n# Use Aer's qasm_simulator\nbackend_sim = Aer.get_backend('qasm_simulator')\n\n# Execute the circuit on the qasm simulator.\n# We've set the number of repeats of the circuit\n# to be 1024, which is the default.\njob_sim = execute(qc, backend_sim, shots=1024)\n\n# Grab the results from the job.\nresult_sim = job_sim.result()\n\ncounts = result_sim.get_counts(qc)\

In [4]:
def substract1(circuit,qubit_string,control,start,end):
    '''
    Outputs:
    Substracts register 2 (1 qubit) from register 1 in register 1. Tested ok.
    
    Input:
    circuit: QuantumCircuit with registers qubit_string, control, ancilla
    
    qubit_string: QuantumRegister
    
    control: Qubit. Use ancilla[0] or similar
    
    start: Qubit. Use ancilla[1] or similar
    end: Qubit. Use ancilla[2] or similar
    
    Comments: In binary, substracting is the same procedure as summing when we exchange 0s and 1s
    '''
    circuit.x(qubit_string)

    sum1(circuit,qubit_string,control,start,end)
    
    circuit.x(qubit_string)
    

'''
# Validation circuit. ------------------------------------------
qubit_string = QuantumRegister(4)
#control = QuantumRegister(1)
ancilla = QuantumRegister(3)

bit_string = ClassicalRegister(4)
#ccontrol = ClassicalRegister(1)
cancilla = ClassicalRegister(3)

qc = QuantumCircuit(qubit_string, ancilla, bit_string, cancilla)

# Initialize state
qc.x(ancilla[0])
#qc.x(qubit_string)

# Gate
substract1(qc,qubit_string,ancilla[0],ancilla[1],ancilla[2])

# Readout
#Measure
qc.measure(qubit_string,bit_string)
qc.measure(ancilla[0],cancilla[0])
qc.measure(ancilla[1],cancilla[1])
qc.measure(ancilla[2],cancilla[2])

# Use Aer's qasm_simulator
backend_sim = Aer.get_backend('qasm_simulator')

# Execute the circuit on the qasm simulator.
# We've set the number of repeats of the circuit
# to be 1024, which is the default.
job_sim = execute(qc, backend_sim, shots=1024)

# Grab the results from the job.
result_sim = job_sim.result()

counts = result_sim.get_counts(qc)

counts
'''

"\n# Validation circuit. ------------------------------------------\nqubit_string = QuantumRegister(4)\n#control = QuantumRegister(1)\nancilla = QuantumRegister(3)\n\nbit_string = ClassicalRegister(4)\n#ccontrol = ClassicalRegister(1)\ncancilla = ClassicalRegister(3)\n\nqc = QuantumCircuit(qubit_string, ancilla, bit_string, cancilla)\n\n# Initialize state\nqc.x(ancilla[0])\n#qc.x(qubit_string)\n\n# Gate\nsubstract1(qc,qubit_string,ancilla[0],ancilla[1],ancilla[2])\n\n# Readout\n#Measure\nqc.measure(qubit_string,bit_string)\nqc.measure(ancilla[0],cancilla[0])\nqc.measure(ancilla[1],cancilla[1])\nqc.measure(ancilla[2],cancilla[2])\n\n# Use Aer's qasm_simulator\nbackend_sim = Aer.get_backend('qasm_simulator')\n\n# Execute the circuit on the qasm simulator.\n# We've set the number of repeats of the circuit\n# to be 1024, which is the default.\njob_sim = execute(qc, backend_sim, shots=1024)\n\n# Grab the results from the job.\nresult_sim = job_sim.result()\n\ncounts = result_sim.get_counts(

In [5]:
def int_angle_func(angle,out_bits):
    out_str = ''
    a = angle
    for bits in range(1,out_bits+1):

        if a +1e-10 > 1/(2**(bits)):
            out_str += '1'
            a -= 1/(2**bits)
        else:
            out_str += '0'
    
    return out_str


'''
angle = .0
out_bits = 8
int_angle = format(int(angle*2**out_bits), 'b')
print(int_angle)
boolean = int_angle_func(angle,out_bits)
print(boolean)
'''

"\nangle = .0\nout_bits = 8\nint_angle = format(int(angle*2**out_bits), 'b')\nprint(int_angle)\nboolean = int_angle_func(angle,out_bits)\nprint(boolean)\n"

In [6]:
class beta_precalc_TruthTableOracle(TruthTableOracle):
    '''Outputs the binary angle of rotation to get the correct probability. Tested ok'''
    def __init__(self, energies_dictionary, beta, out_bits, optimization=True, mct_mode='noancilla'):
        self.beta = beta
        self.out_bits = out_bits
        self.in_bits = len(list(energies_dictionary.keys())[0]) # The key of energies_dictionary is the input bits
        self.energies_dictionary = energies_dictionary
        #print(self.energies_dictionary)
        # print('Is parameter',self.parameters) --- Deprecated
        self.calculate_bitmap()
        super().__init__(self.bitmap, optimization, mct_mode)
        
    def calculate_bitmap(self):
        new_bitmap = []
        angles = {}
        for i in range(int(2**self.in_bits)):
            st = '0'*(self.in_bits - len(str(format(i,'b')))) + str(format(i,'b'))
            #print(st)
            if self.energies_dictionary[st] >= 0:
                #print(type(np))
                probability = math.exp(-self.beta * self.energies_dictionary[st])
                #print('probability: ',probability)
                #print('energy: ', self.energies_dictionary[st])
                #print('beta:', self.beta)
            else: 
                probability = 1
                #print('probability = 1')
                
            # Instead of encoding the probability, we will encode 1-probability. That way 1 -> 000, 
            #but if probability is 0 there is some small probability of acceptance
            probability = 1 - probability
            
            
            # Instead of probability save angles so rotations are easier to perform afterwards sqrt(p) = sin(theta)
            angle = math.asin(math.sqrt(probability))
            
            # Make the angle be between [0,1]. Since the maximum is pi/2
            angle /= (math.pi/2) 
            
            # Convert it into an integer and a string
            #int_angle = format(int(angle*2**self.out_bits), 'b')
            #str_angle = str(int_angle) 
            str_angle2 = int_angle_func(angle,self.out_bits)
            # Convert it to binary
            #int_angle = format(int(angle*2**out_bits), 'b')
            '''
            if int_angle == '1' + '0'*out_bits:
                angles[st] = '1'*out_bits # As we only have out_bits, the 10000 is substituted by 1111
            else:
                str_angle = str(int_angle)
                angles[st] = '0'*(out_bits - len(str_angle)) + str_angle
            ''' 
            angles[st] = str_angle2
            #print('dict_key',st)
            #print('energies',self.energies_dictionary[st])
            #print('probability',probability)
            #print('angle',angle)
            #print('angles', angles[st])
        
        # Encoding the new bitmap
        new_bitmap = []
        for o in range(self.out_bits):
            string = ''
            for i in range(int(2**self.in_bits)):
                st = '0'*(self.in_bits - len(str(format(i,'b')))) + str(format(i,'b'))
                string += str(angles[st])[o]
            new_bitmap += [string]
        #print('angles',angles)
        #print('bitmap',new_bitmap)
        self.bitmap = new_bitmap

'''        
# Validation -----------------
energies_dictionary = {}
beta = 1

# Lets set 2 bits for phi and 2 for phi. Lets create a dictionary of random energies.
energies = {}
for i in range(16):
    key = str(format(i, 'b'))
    key = '0'*(4 - len(key)) + key
    energies[key] = np.random.random()

#print(energies)
    
for key in energies.keys():
    for angle_select in ('0','1'):
        for plus_minus in ('0','1'):
            
            phi = int(key[:2])
            psi = int(key[2:])
            angle_s = int(angle_select)
            p_s = 2*int(plus_minus)-1
            
            phi_n = (phi + p_s*(1-angle_s) + 4) % 4 # The (x +4 )%4 ensures the number is positive
            psi_n = (psi + p_s*(angle_s) + 4) % 4
            phi_s = str(format(int(phi_n),'b'))
            psi_s = str(format(int(psi_n),'b'))
            phi_s = '0'*(2 - len(phi_s)) + phi_s
            psi_s = '0'*(2 - len(psi_s)) + psi_s
            
            new_key = phi_s + psi_s
            
            e_old = energies[key]
            e_new = energies[new_key]
            
            energies_dictionary[key + angle_select + plus_minus] = e_new - e_old

            
print('energies_dictionary')
print(energies_dictionary)            
            
out_bits = 6            

oracle = beta_precalc_TruthTableOracle(energies_dictionary, beta, out_bits)

print('bitmap')

print(oracle.bitmap)
'''

"        \n# Validation -----------------\nenergies_dictionary = {}\nbeta = 1\n\n# Lets set 2 bits for phi and 2 for phi. Lets create a dictionary of random energies.\nenergies = {}\nfor i in range(16):\n    key = str(format(i, 'b'))\n    key = '0'*(4 - len(key)) + key\n    energies[key] = np.random.random()\n\n#print(energies)\n    \nfor key in energies.keys():\n    for angle_select in ('0','1'):\n        for plus_minus in ('0','1'):\n            \n            phi = int(key[:2])\n            psi = int(key[2:])\n            angle_s = int(angle_select)\n            p_s = 2*int(plus_minus)-1\n            \n            phi_n = (phi + p_s*(1-angle_s) + 4) % 4 # The (x +4 )%4 ensures the number is positive\n            psi_n = (psi + p_s*(angle_s) + 4) % 4\n            phi_s = str(format(int(phi_n),'b'))\n            psi_s = str(format(int(psi_n),'b'))\n            phi_s = '0'*(2 - len(phi_s)) + phi_s\n            psi_s = '0'*(2 - len(psi_s)) + psi_s\n            \n            new_key = phi

## Quantum circuit steps

Let us first define the quantum circuit

In [7]:
# This is the move preparation gate
def move_preparation(circuit,move_id,move_value):
    '''
    Proposes new moves
    '''
    circuit.h(move_id)
    circuit.h(move_value)

# Let us define it as a portable gate
s_move_id = QuantumRegister(1) 
s_move_value = QuantumRegister(1)

sub_circ = QuantumCircuit(s_move_id, s_move_value)

move_preparation(sub_circ,s_move_id,s_move_value)

move_preparation = sub_circ.to_instruction()


# Use as qc.append(move_preparation, [move_id[0], move_value[0]])

Next, we define the conditional move. For that we have to go over all possible combinations of move_id and move_value, adding or substracting 1, conditioned on the coin being at state 1.


In [8]:
 def conditional_move_dipeptide(circuit,coin,move_id,move_value,angle_phi,angle_psi,ancilla):
    '''
    Conditioned on coin, perform a move. For a dipeptide!
    We use a repetitive structure where we perform the conditional sum and substraction for each angle.
    Checked ok
    '''
    # For angle_phi = angle_id = 0 ----------------------------------------------
    circuit.x(move_id) # Put move_id in 1
    
    # Conditional on move_id = 0, move_value = 1 and coin = 1, increase angle_phi by on
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_value[0], coin[0]], q_target = ancilla[0])
    #circuit.append(Gate("mcx", 5, []), [move_id[0], move_id[1], move_value, coin, ancilla[0]]) #create a single control
    sum1(circuit,angle_phi,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_value[0], coin[0]], q_target = ancilla[0])
    
    # Conditional on move_id = 0, move_value = 0 and coin = 1, decrease angle_ psi by on
    # Put move_id in 11
    circuit.x(move_value)
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    substract1(circuit,angle_phi,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    circuit.x(move_value)
    
    circuit.x(move_id)
    
    # For angle_psi = angle_id = 1 ----------------------------------------------
    # Put move_id in 11
    
    # Conditional on move_id = 1, move_value = 1 and coin = 1, increase angle_psi by one
    circuit.mcrx(theta = pi, q_controls = [move_id[0],move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    sum1(circuit,angle_psi,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0],move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    
    # Conditional on move_id = 1, move_value = 0 and coin = 1, decrease angle_psi by one
    # Put move_id in 11
    circuit.x(move_value)
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    substract1(circuit,angle_psi,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    circuit.x(move_value)
    
    
# Let us define it as a portable gate
s_move_id = QuantumRegister(1) 
s_move_value = QuantumRegister(1)
s_coin = QuantumRegister(1)
s_ancilla = QuantumRegister(n_ancilla_bits)
s_angle_phi = QuantumRegister(n_precision_bits, name = 'angle_phi')
s_angle_psi = QuantumRegister(n_precision_bits, name = 'angle_psi') 


sub_circ = QuantumCircuit(s_angle_phi,s_angle_psi,s_move_id,s_move_value,s_coin,s_ancilla)

conditional_move_dipeptide(sub_circ,s_coin,s_move_id,s_move_value,s_angle_phi,s_angle_psi,s_ancilla)
conditional_move = sub_circ.to_instruction()

# To use it or its inverse
# qc.append(conditional_move.inverse(), [(angle_phi[j] for j in range(angle_phi.size)),(angle_psi[j] for j in range(angle_psi.size)),move_id[0], move_value[0], coin[0], ancilla[0], ancilla[1], ancilla[2]])

'''
# Validation circuit. ------------------------------------------
coin = QuantumRegister(1)
move_id = QuantumRegister(1)
move_value = QuantumRegister(1)
angle_phi = QuantumRegister(3)
angle_psi = QuantumRegister(3)
ancilla = QuantumRegister(3)

ccoin = ClassicalRegister(1)
cmove_id = ClassicalRegister(1)
cmove_value = ClassicalRegister(1)
cangle_phi = ClassicalRegister(3)
cangle_psi = ClassicalRegister(3)
cancilla = ClassicalRegister(3)

qc = QuantumCircuit(coin,move_id, move_value,angle_phi, angle_psi,ancilla, ccoin, cmove_id, cmove_value, cangle_phi, cangle_psi, cancilla)

# Initialize state
qc.x(coin)
qc.x(angle_phi[0])
#qc.x(qubit_string)

# Gate
conditional_move_dipeptide(qc,coin,move_id,move_value,angle_phi,angle_psi,ancilla)

# Readout
#Measure
qc.measure(coin,ccoin)
qc.measure(move_id,cmove_id)
qc.measure(move_value,cmove_value)
qc.measure(angle_phi, cangle_phi)
qc.measure(angle_psi, cangle_psi)
qc.measure(ancilla,cancilla)

# Use Aer's qasm_simulator
backend_sim = Aer.get_backend('qasm_simulator')

# Execute the circuit on the qasm simulator.
# We've set the number of repeats of the circuit
# to be 1024, which is the default.
job_sim = execute(qc, backend_sim, shots=1024)

# Grab the results from the job.
result_sim = job_sim.result()

counts = result_sim.get_counts(qc)

counts
'''

"\n# Validation circuit. ------------------------------------------\ncoin = QuantumRegister(1)\nmove_id = QuantumRegister(1)\nmove_value = QuantumRegister(1)\nangle_phi = QuantumRegister(3)\nangle_psi = QuantumRegister(3)\nancilla = QuantumRegister(3)\n\nccoin = ClassicalRegister(1)\ncmove_id = ClassicalRegister(1)\ncmove_value = ClassicalRegister(1)\ncangle_phi = ClassicalRegister(3)\ncangle_psi = ClassicalRegister(3)\ncancilla = ClassicalRegister(3)\n\nqc = QuantumCircuit(coin,move_id, move_value,angle_phi, angle_psi,ancilla, ccoin, cmove_id, cmove_value, cangle_phi, cangle_psi, cancilla)\n\n# Initialize state\nqc.x(coin)\nqc.x(angle_phi[0])\n#qc.x(qubit_string)\n\n# Gate\nconditional_move_dipeptide(qc,coin,move_id,move_value,angle_phi,angle_psi,ancilla)\n\n# Readout\n#Measure\nqc.measure(coin,ccoin)\nqc.measure(move_id,cmove_id)\nqc.measure(move_value,cmove_value)\nqc.measure(angle_phi, cangle_phi)\nqc.measure(angle_psi, cangle_psi)\nqc.measure(ancilla,cancilla)\n\n# Use Aer's qasm_

The equivalent cell for tripeptides is

In [9]:
'''
def conditional_move_tripeptide(circuit,coin,move_id,move_value,angle_id,angle_value,ancilla):
'''
#Conditioned on coin, perform a move. For a tripeptide!
#We use a repetitive structure where we perform the conditional sum and substraction for each angle.
'''
    # For angle_0 ----------------------------------------------
    circuit.x(move_id) # Put move_id in 11
    
    # Conditional on move_id = 00, move_value = 1 and coin = 1, increase angle_0 by on
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])
    #circuit.append(Gate("mcx", 5, []), [move_id[0], move_id[1], move_value, coin, ancilla[0]]) #create a single control
    sum1(circuit,angle_0,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])
    
    # Conditional on move_id = 00, move_value = 0 and coin = 1, decrease angle_0 by on
    # Put move_id in 11
    circuit.x(move_value)
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    substract1(circuit,angle_0,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    circuit.x(move_value)
    
    circuit.x(move_id)
    
    # For angle_1 ----------------------------------------------
    circuit.x(move_id[1]) # Put move_id in 11
    
    # Conditional on move_id = 00, move_value = 1 and coin = 1, increase angle_0 by on
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1],move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    sum1(circuit,angle_1,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1],move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    
    # Conditional on move_id = 00, move_value = 0 and coin = 1, decrease angle_0 by on
    # Put move_id in 11
    circuit.x(move_value)
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    substract1(circuit,angle_1,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    circuit.x(move_value)
    
    circuit.x(move_id[1])
    
    # For angle_2 ----------------------------------------------
    circuit.x(move_id[0]) # Put move_id in 11
    
    # Conditional on move_id = 00, move_value = 1 and coin = 1, increase angle_0 by on
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    sum1(circuit,angle_2,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    
    # Conditional on move_id = 00, move_value = 0 and coin = 1, decrease angle_0 by on
    # Put move_id in 11
    circuit.x(move_value)
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    substract1(circuit,angle_2,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    circuit.x(move_value)
    
    circuit.x(move_id[0])
    
    # For angle_3 ----------------------------------------------
    #circuit.x(move_id[0]) #  move_id  is in 11
    
    # Conditional on move_id = 00, move_value = 1 and coin = 1, increase angle_0 by on
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    sum1(circuit,angle_3,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    
    # Conditional on move_id = 00, move_value = 0 and coin = 1, decrease angle_0 by on
    # Put move_id in 11
    circuit.x(move_value)
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    substract1(circuit,angle_3,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) = 6 in this cased
    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control
    circuit.x(move_value)
    
    #circuit.x(move_id[0])
    
# Let us define it as a portable gate
s_move_id = QuantumRegister(move_id.size) 
s_move_value = QuantumRegister(move_value.size)
s_coin = QuantumRegister(coin.size)
s_ancilla = QuantumRegister(ancilla.size)
s_angle_0 = QuantumRegister(5, name = 'angle_0')
s_angle_1 = QuantumRegister(5, name = 'angle_1') 
s_angle_2 = QuantumRegister(5, name = 'angle_2') 
s_angle_3 = QuantumRegister(5, name = 'angle_3') 

sub_circ = QuantumCircuit(s_angle_0,s_angle_1,s_angle_2,s_angle_3,s_move_id,s_move_value,s_coin,s_ancilla)
conditional_move_tripeptide(sub_circ,s_coin,s_move_id,s_move_value,s_angle_id,s_angle_value,s_ancilla)
conditional_move = sub_circ.to_instruction()

qc.append(conditional_move.inverse(), [(angle_0[j] for j in range(4)),(angle_1[j] for j in range(4)),(angle_2[j] for j in range(4)),(angle_3[j] for j in range(4)),move_id[0], move_id[1], move_value[0],coin[0],ancilla[0],ancilla[1],ancilla[2]])
'''

'\n    # For angle_0 ----------------------------------------------\n    circuit.x(move_id) # Put move_id in 11\n    \n    # Conditional on move_id = 00, move_value = 1 and coin = 1, increase angle_0 by on\n    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])\n    #circuit.append(Gate("mcx", 5, []), [move_id[0], move_id[1], move_value, coin, ancilla[0]]) #create a single control\n    sum1(circuit,angle_0,ancilla[0],ancilla[1],ancilla[2]) #calc_anc_size(len_angle_value) =6 in this cased\n    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])\n    \n    # Conditional on move_id = 00, move_value = 0 and coin = 1, decrease angle_0 by on\n    # Put move_id in 11\n    circuit.x(move_value)\n    circuit.mcrx(theta = pi, q_controls = [move_id[0], move_id[1], move_value[0], coin[0]], q_target = ancilla[0])#create a single control\n    substract1(circuit,angle_0,ancilla[0],an

Define the reflection around 0

In [10]:
def reflection(circuit,move_id,move_value,coin):
    '''
    I have to investigate over what is the reflection performed. Is it performed over 000?
    If in state 0000, make it 1111, cccz gate and back to 0000
    '''
    circuit.x(move_id)
    circuit.x(move_value)
    circuit.x(coin)
    
    circuit.mcrz(lam = pi, q_controls = [move_id[0]] + [move_value[0]], q_target = coin[0]) #For dipeptide
    # circuit.mcrz(lam = pi, q_controls = [move_id[0]] + [move_id[1]]+ [move_value[0]], q_target = coin[0]) #For tripeptide
    
    circuit.x(move_id)
    circuit.x(move_value)
    circuit.x(coin)
    

# Let us define it as a portable gate
s_move_id = QuantumRegister(1) 
s_move_value = QuantumRegister(1)
s_coin = QuantumRegister(1)


sub_circ = QuantumCircuit(s_move_id,s_move_value,s_coin)
reflection(sub_circ,s_move_id,s_move_value,s_coin)
reflection = sub_circ.to_instruction()

# Use as qc.append(reflection.inverse(), [move_id[0], move_value[0],coin[0]])

We now define the coin flip, which takes three steps:

1.    Calculating the oracle that outputs the probability encoded in the ancilla
2.    Performing the coin flip
3.    Undoing the oracle to uncompute the ancilla

We create a subcircuit sub_circ with registers of type 's_ ' where we perform all the calculations. Unfortunately, we will have to go over all this next lines in order to perform the entire calculation, which cannot be packed in a gate due to the dependence to the beta parameter.

Notice that the main difficulty is building a parametrised instruction.

In [11]:
def coin_flip(circuit,coin,ancilla):
    '''
    Prepares the coin with the probability encoded in the ancilla.
    coin: register[0]
    ancilla: register[1:]
    '''
    
    #Necesitamos usar el número guardado en las ancillas para realizar rotaciones controladas.  
    #Notice that ancilla encodes 1-probability, rather than probability.
    #Notice also that cu3(theta) rotates theta/2. As the first angle to rotate is pi/4 we need to start in theta = pi/2

    circuit.x(coin) # Start in 1 and decrease it, since we encoded the angle corresponding 1-probability
    for i in range(ancilla.size):
        circuit.cu3(theta = -math.pi/(2**(i+1)), phi  = 0, lam = 0, control_qubit = ancilla[i], target_qubit = coin)
    

In [12]:
'''
# Validation circuit. ------------------------------------------
coin = QuantumRegister(1)
ancilla = QuantumRegister(3)

ccoin = ClassicalRegister(1)
cancilla = ClassicalRegister(3)

qc = QuantumCircuit(coin,ancilla, ccoin, cancilla)

# Initialize state
qc.x(ancilla)
#qc.x(qubit_string)

# Gate
coin_flip(qc,coin,ancilla)

# Readout
#Measure
qc.measure(coin,ccoin)
qc.measure(ancilla,cancilla)

# Use Aer's qasm_simulator
backend_sim = Aer.get_backend('qasm_simulator')

# Execute the circuit on the qasm simulator.
# We've set the number of repeats of the circuit
# to be 1024, which is the default.
job_sim = execute(qc, backend_sim, shots=4096)

# Grab the results from the job.
result_sim = job_sim.result()

counts = result_sim.get_counts(qc)

print(counts)


angle = np.pi/2*(.5 + .5/2 + .5/4)
print(np.cos(angle)**2)
print(166/4096)
'''

"\n# Validation circuit. ------------------------------------------\ncoin = QuantumRegister(1)\nancilla = QuantumRegister(3)\n\nccoin = ClassicalRegister(1)\ncancilla = ClassicalRegister(3)\n\nqc = QuantumCircuit(coin,ancilla, ccoin, cancilla)\n\n# Initialize state\nqc.x(ancilla)\n#qc.x(qubit_string)\n\n# Gate\ncoin_flip(qc,coin,ancilla)\n\n# Readout\n#Measure\nqc.measure(coin,ccoin)\nqc.measure(ancilla,cancilla)\n\n# Use Aer's qasm_simulator\nbackend_sim = Aer.get_backend('qasm_simulator')\n\n# Execute the circuit on the qasm simulator.\n# We've set the number of repeats of the circuit\n# to be 1024, which is the default.\njob_sim = execute(qc, backend_sim, shots=4096)\n\n# Grab the results from the job.\nresult_sim = job_sim.result()\n\ncounts = result_sim.get_counts(qc)\n\nprint(counts)\n\n\nangle = np.pi/2*(.5 + .5/2 + .5/4)\nprint(np.cos(angle)**2)\nprint(166/4096)\n"

Previous result
{'111 1': 171, '111 0': 3925}
0.03806023374435664
0.04052734375

Given the special nature of the parameter beta, we cannot write the following function in the same function as we did previously. Coin_flip_func will make use of 

In [13]:
def coin_flip_func(oracle):
    
    '''
    Defines de coin_flip_gate using the oracle that is provided on the moment.
    Notice that oracle gate has registers oracle.variable_register and oracle.output_register in that order
    oracle.variable_register should have size angle_phi.size + angle_psi.size + move_id.size + move_value.size
    oracle.output_register should have size ancilla.size
    '''

    # Construct an instruction for the oracle
    oracle.construct_circuit()
    oracle_circuit = oracle.circuit
    #print(oracle_circuit)
    oracle_gate = oracle_circuit.to_instruction()


    # Let us create a circuit for coin_flip
    cf_move_id = QuantumRegister(1) 
    cf_move_value = QuantumRegister(1)
    cf_coin = QuantumRegister(1)
    cf_ancilla = QuantumRegister(n_ancilla_bits)
    cf_angle_phi = QuantumRegister(n_precision_bits, name = 'angle_phi')
    cf_angle_psi = QuantumRegister(n_precision_bits, name = 'angle_psi') 

    cf_circ = QuantumCircuit(cf_angle_phi, cf_angle_psi, cf_move_id, cf_move_value ,cf_coin , cf_ancilla)

    # Main operations
    cf_circ.append(oracle_gate, [cf_angle_phi[j] for j in range(cf_angle_phi.size)]+[cf_angle_psi[j] for j in range(cf_angle_psi.size)]+ [cf_move_id[0], cf_move_value[0]] +[cf_ancilla[j] for j in range(cf_ancilla.size)])
    coin_flip(cf_circ,cf_coin,cf_ancilla)
    cf_circ.append(oracle_gate.inverse(), [cf_angle_phi[j] for j in range(cf_angle_phi.size)]+[cf_angle_psi[j] for j in range(cf_angle_psi.size)]+ [cf_move_id[0], cf_move_value[0]] +[cf_ancilla[j] for j in range(cf_ancilla.size)])

    coin_flip_gate = cf_circ.to_instruction()
    
    return coin_flip_gate



'''
Use as:
    #coin_flip_gate.params[0]= a_given_beta
    qc.append(coin_flip_gate.inverse(), [(angle_phi[j] for j in range(angle_phi.size)),angle_psi[j] for j in range(angle_psi.size)),move_id[0], move_value[0],coin[0],(ancilla[j] for j in range(ancilla.size))])
'''

'\nUse as:\n    #coin_flip_gate.params[0]= a_given_beta\n    qc.append(coin_flip_gate.inverse(), [(angle_phi[j] for j in range(angle_phi.size)),angle_psi[j] for j in range(angle_psi.size)),move_id[0], move_value[0],coin[0],(ancilla[j] for j in range(ancilla.size))])\n'

## Define the gate implementing W and the main loop.

In [18]:
def W_func(oracle):
    
    '''This defines the parametrised gate W using the oracle that is provided to it, and we can reuse its inverse too.'''

    # State definition. All angles range from 0 to 2pi
    w_angle_phi = QuantumRegister(n_precision_bits, name = 'angle_phi')
    w_angle_psi = QuantumRegister(n_precision_bits, name = 'angle_psi') 

    # Move proposal
    w_move_id = QuantumRegister(1, name = 'move_id') #Which angle are we modifying
    w_move_value = QuantumRegister(1, name = 'move_value') #0 -> decrease the angle. 1-> increase it

    # Coin
    w_coin = QuantumRegister(1, name = 'coin')

    # Ancillas
    w_ancilla = QuantumRegister(n_ancilla_bits, name = 'ancilla')

    # Circuit
    qc = QuantumCircuit(w_angle_phi,w_angle_psi,w_move_id,w_move_value,w_coin,w_ancilla)

    #D beta = Parameter('β') ---- Deprecated
    #D coin_flip_gate.params[0]= beta ---- Deprecated
    
    # Define the coin_flip_gate
    coin_flip_gate = coin_flip_func(oracle)

    # Move preparation
    qc.append(move_preparation, [w_move_id[0], w_move_value[0]])
    
    # Coin flip    
    qc.append(coin_flip_gate, [w_angle_phi[j] for j in range(w_angle_phi.size)]+[w_angle_psi[j] for j in range(w_angle_psi.size)] + [w_move_id[0], w_move_value[0],w_coin[0]] + [w_ancilla[j] for j in range(w_ancilla.size)])

    # Conditional move
    qc.append(conditional_move, [w_angle_phi[j] for j in range(w_angle_phi.size)]+[w_angle_psi[j] for j in range(w_angle_psi.size)] + [w_move_id[0], w_move_value[0], w_coin[0]] + [w_ancilla[j] for j in range(w_ancilla.size)])

    # Inverse coin flip
    qc.append(coin_flip_gate.inverse(), [w_angle_phi[j] for j in range(w_angle_phi.size)]+[w_angle_psi[j] for j in range(w_angle_psi.size)] + [w_move_id[0], w_move_value[0],w_coin[0]] + [w_ancilla[j] for j in range(w_ancilla.size)])

    # Inverse move preparation
    qc.append(move_preparation, [w_move_id[0], w_move_value[0]])

    # Reflection
    qc.append(reflection, [w_move_id[0], w_move_value[0],w_coin[0]])

    W_gate = qc.to_instruction()
    
    return W_gate

'''
Use as:
    #W_gate.params[0]= a_given_beta
    qc.append(W_gate.inverse(), [(angle_psi[j] for j in range(angle_psi.size)),angle_phi[j] for j in range(angle_phi.size)),move_id[0], move_value[0],coin[0],(ancilla[j] for j in range(ancilla.size))])
'''

'\nUse as:\n    #W_gate.params[0]= a_given_beta\n    qc.append(W_gate.inverse(), [(angle_psi[j] for j in range(angle_psi.size)),angle_phi[j] for j in range(angle_phi.size)),move_id[0], move_value[0],coin[0],(ancilla[j] for j in range(ancilla.size))])\n'

In [16]:
import json
from itertools import product

with open('precalculated_energies/energies_glycylglycine_3.json') as json_file:
    data = json.load(json_file)

assert(data['numberBitsRotation'] == n_precision_bits)
energies = data['energies']

# Let us create the energy dictionary        
energies_dictionary0 = {}

for item in energies:
    
    phi = format(int(item['phi']),'b')
    phi = '0'*(n_precision_bits - len(phi)) + phi
    psi = format(int(item['psi']),'b')
    psi = '0'*(n_precision_bits - len(psi)) + psi
    energy = item['energy']
    energies_dictionary0[phi + psi] = energy
    
    
energies_dictionary = {}
for phi, psi in product(range(2**n_precision_bits),range(2**n_precision_bits)):    
    int_phi = format(phi,'b')
    int_phi = '0'*(n_precision_bits - len(int_phi)) + int_phi
    int_psi = format(psi,'b')
    int_psi = '0'*(n_precision_bits - len(int_psi)) + int_psi
    old_E = energies_dictionary0[int_phi + int_psi]
    
    for plusminus in [0,1]:
        pm = 2*plusminus - 1
        
        for phipsi in [0,1]:
            if phipsi == 0:
                new_phi = (phi + pm) % (2**n_precision_bits)
                new_psi = psi
                
            if phipsi == 1:
                new_phi = phi
                new_psi = (psi + pm) % (2**n_precision_bits)
                
            int_new_phi = format(new_phi,'b')
            int_new_phi = '0'*(n_precision_bits - len(int_new_phi)) + int_new_phi
            int_new_psi = format(new_psi,'b')
            int_new_psi = '0'*(n_precision_bits - len(int_new_psi)) + int_new_psi
            new_E = energies_dictionary0[int_new_phi + int_new_psi]
            
            energies_dictionary[int_phi + int_psi + str(phipsi) + str(plusminus)] = new_E - old_E
            

#print(energies_dictionary)

In [22]:
# State definition. All angles range from 0 to 2pi
g_angle_phi = QuantumRegister(n_precision_bits, name = 'angle_phi')
g_angle_psi = QuantumRegister(n_precision_bits, name = 'angle_psi') 

# Move proposal
g_move_id = QuantumRegister(1, name = 'move_id') #Which angle are we modifying
g_move_value = QuantumRegister(1, name = 'move_value') #0 -> decrease the angle. 1-> increase it

# Coin
g_coin = QuantumRegister(1, name = 'coin')

# Ancillas
g_ancilla = QuantumRegister(n_ancilla_bits, name = 'ancilla')

# Circuit
qc = QuantumCircuit(g_angle_phi,g_angle_psi,g_move_id,g_move_value,g_coin,g_ancilla)

# Define number of steps
L=1
beta_max = 1
# Read energies_dictionary

# Metropolis algorithm
#lista = []
for i in range(L):
    beta = (1+i)/L*beta_max
    
    oracle = beta_precalc_TruthTableOracle(energies_dictionary,beta,out_bits = n_ancilla_bits)
    
    W_gate = W_func(oracle)
    
    #lista.append(deepcopy(W_gate)) # We deepcopy W_gate to not interfere with other calls
    #lista[i].params[0]= beta
    qc.append(W_gate, [g_angle_phi[j] for j in range(g_angle_phi.size)] + [g_angle_psi[j] for j in range(g_angle_psi.size)] + [g_move_id[0], g_move_value[0],g_coin[0]] + [g_ancilla[j] for j in range(g_ancilla.size)])


''' 
#Old version ---- Deprecated
for i in range(L):
    W_gate.params[0] = i/L*beta_max  # Warning!! Does it change the beta for all W already in the circuit? If so, deepcopy?
    circuit.append(W_gate, [(g_angle_phi[j] for j in range(g_angle_phi.size)),(g_angle_psi[j] for j in range(g_angle_psi.size)),g_move_id[0], g_move_value[0],g_coin[0],(g_ancilla[j] for j in range(g_ancilla.size))])
'''
    
# Add measurements
#qc.barrier(range(32))


phi = ClassicalRegister(g_angle_phi.size)
psi = ClassicalRegister(g_angle_psi.size)
qc += QuantumCircuit(phi,psi)

# map the quantum measurement to the classical bits
qc.measure(g_angle_phi,phi)
qc.measure(g_angle_psi,psi)
print('results')
# Execute the circuit
backend = BasicAer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=1024) # Start with that and move to 4096
job.result().get_counts(qc)

results


{'000 000': 82, '000 001': 261, '111 000': 263, '000 111': 238, '001 000': 180}